In [62]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

 Data Loading
 

In [2]:
PSL_9 = pd.read_csv("PSL_9.csv")
print(PSL_9)

      match_id   season  start_date                      venue  innings  ball  \
0      1416472  2023/24  2024-02-17    Gaddafi Stadium, Lahore        1   1.5   
1      1416472  2023/24  2024-02-17    Gaddafi Stadium, Lahore        1   1.6   
2      1416472  2023/24  2024-02-17    Gaddafi Stadium, Lahore        1   2.1   
3      1416472  2023/24  2024-02-17    Gaddafi Stadium, Lahore        1   2.2   
4      1416472  2023/24  2024-02-17    Gaddafi Stadium, Lahore        1   2.3   
...        ...      ...         ...                        ...      ...   ...   
7790   1416505  2023/24  2024-03-18  National Stadium, Karachi        2  19.2   
7791   1416505  2023/24  2024-03-18  National Stadium, Karachi        2  19.3   
7792   1416505  2023/24  2024-03-18  National Stadium, Karachi        2  19.4   
7793   1416505  2023/24  2024-03-18  National Stadium, Karachi        2  19.5   
7794   1416505  2023/24  2024-03-18  National Stadium, Karachi        2  19.6   

          batting_team     

In [5]:
PSL_9.head(2)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,1416472,2023/24,2024-02-17,"Gaddafi Stadium, Lahore",1,1.5,Lahore Qalandars,Islamabad United,Sahibzada Farhan,Fakhar Zaman,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1416472,2023/24,2024-02-17,"Gaddafi Stadium, Lahore",1,1.6,Lahore Qalandars,Islamabad United,Sahibzada Farhan,Fakhar Zaman,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Data Exploration

In [8]:
PSL_9.shape

(7795, 22)

In [12]:
PSL_9.dtypes

match_id                    int64
season                     object
start_date                 object
venue                      object
innings                     int64
ball                      float64
batting_team               object
bowling_team               object
striker                    object
non_striker                object
bowler                     object
runs_off_bat                int64
extras                      int64
wides                     float64
noballs                   float64
byes                      float64
legbyes                   float64
penalty                   float64
wicket_type                object
player_dismissed           object
other_wicket_type         float64
other_player_dismissed    float64
dtype: object

In [14]:
PSL_9.describe()

,match_id,innings,ball,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,other_wicket_type,other_player_dismissed
count,7.795000e+03,7795.000000,7795.000000,7795.000000,7795.000000,288.000000,23.0,16.000000,136.000000,1.0,0.0,0.0
mean,1.416488e+06,1.489416,9.695548,1.322001,0.075690,1.229167,1.0,2.250000,1.264706,5.0,NaN,NaN
std,1.011414e+01,0.499920,5.694842,1.650024,0.367276,0.828236,0.0,1.437591,0.800327,NaN,NaN,NaN
min,1.416472e+06,1.000000,0.100000,0.000000,0.000000,1.000000,1.0,1.000000,1.000000,5.0,NaN,NaN
25%,1.416479e+06,1.000000,4.600000,0.000000,0.000000,1.000000,1.0,1.000000,1.000000,5.0,NaN,NaN
50%,1.416487e+06,1.000000,9.600000,1.000000,0.000000,1.000000,1.0,1.500000,1.000000,5.0,NaN,NaN
75%,1.416497e+06,2.000000,14.500000,1.000000,0.000000,1.000000,1.0,4.000000,1.000000,5.0,NaN,NaN
max,1.416505e+06,2.000000,19.900000,6.000000,5.000000,5.000000,1.0,4.000000,4.000000,5.0,NaN,NaN


In [15]:
PSL_9.isnull().sum()

match_id                     0
season                       0
start_date                   0
venue                        0
innings                      0
ball                         0
batting_team                 0
bowling_team                 0
striker                      0
non_striker                  0
bowler                       0
runs_off_bat                 0
extras                       0
wides                     7507
noballs                   7772
byes                      7779
legbyes                   7659
penalty                   7794
wicket_type               7392
player_dismissed          7392
other_wicket_type         7795
other_player_dismissed    7795
dtype: int64

In [20]:
PSL_9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7795 entries, 0 to 7794
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   match_id                7795 non-null   int64  
 1   season                  7795 non-null   object 
 2   start_date              7795 non-null   object 
 3   venue                   7795 non-null   object 
 4   innings                 7795 non-null   int64  
 5   ball                    7795 non-null   float64
 6   batting_team            7795 non-null   object 
 7   bowling_team            7795 non-null   object 
 8   striker                 7795 non-null   object 
 9   non_striker             7795 non-null   object 
 10  bowler                  7795 non-null   object 
 11  runs_off_bat            7795 non-null   int64  
 12  extras                  7795 non-null   int64  
 13  wides                   288 non-null    float64
 14  noballs                 23 non-null     

In [21]:
PSL_9.duplicated().sum()

0

In [23]:
PSL_9.corr

<bound method DataFrame.corr of       match_id   season  start_date                      venue  innings  ball  \
0      1416472  2023/24  2024-02-17    Gaddafi Stadium, Lahore        1   1.5   
1      1416472  2023/24  2024-02-17    Gaddafi Stadium, Lahore        1   1.6   
2      1416472  2023/24  2024-02-17    Gaddafi Stadium, Lahore        1   2.1   
3      1416472  2023/24  2024-02-17    Gaddafi Stadium, Lahore        1   2.2   
4      1416472  2023/24  2024-02-17    Gaddafi Stadium, Lahore        1   2.3   
...        ...      ...         ...                        ...      ...   ...   
7790   1416505  2023/24  2024-03-18  National Stadium, Karachi        2  19.2   
7791   1416505  2023/24  2024-03-18  National Stadium, Karachi        2  19.3   
7792   1416505  2023/24  2024-03-18  National Stadium, Karachi        2  19.4   
7793   1416505  2023/24  2024-03-18  National Stadium, Karachi        2  19.5   
7794   1416505  2023/24  2024-03-18  National Stadium, Karachi        2  19.6

Data Pre-Process


In [26]:
PSL_9.dropna()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed


In [30]:
PSL_9 = pd.get_dummies(PSL_9, drop_first=True)
print(PSL_9)

      match_id  innings  ball  runs_off_bat  extras  wides  noballs  byes  \
0      1416472        1   1.5             2       0    NaN      NaN   NaN   
1      1416472        1   1.6             4       0    NaN      NaN   NaN   
2      1416472        1   2.1             0       5    5.0      NaN   NaN   
3      1416472        1   2.2             1       0    NaN      NaN   NaN   
4      1416472        1   2.3             4       0    NaN      NaN   NaN   
...        ...      ...   ...           ...     ...    ...      ...   ...   
7790   1416505        2  19.2             4       0    NaN      NaN   NaN   
7791   1416505        2  19.3             0       1    NaN      NaN   NaN   
7792   1416505        2  19.4             1       0    NaN      NaN   NaN   
7793   1416505        2  19.5             0       0    NaN      NaN   NaN   
7794   1416505        2  19.6             4       0    NaN      NaN   NaN   

      legbyes  penalty  ...  player_dismissed_T Kohler-Cadmore  \
0        

In [39]:
X = PSL_9.drop(columns='match_id', axis=1)
Y= PSL_9['match_id']


In [40]:
print(X)

      innings  ball  runs_off_bat  extras  wides  noballs  byes  legbyes  \
0           1   1.5             2       0    NaN      NaN   NaN      NaN   
1           1   1.6             4       0    NaN      NaN   NaN      NaN   
2           1   2.1             0       5    5.0      NaN   NaN      NaN   
3           1   2.2             1       0    NaN      NaN   NaN      NaN   
4           1   2.3             4       0    NaN      NaN   NaN      NaN   
...       ...   ...           ...     ...    ...      ...   ...      ...   
7790        2  19.2             4       0    NaN      NaN   NaN      NaN   
7791        2  19.3             0       1    NaN      NaN   NaN      1.0   
7792        2  19.4             1       0    NaN      NaN   NaN      NaN   
7793        2  19.5             0       0    NaN      NaN   NaN      NaN   
7794        2  19.6             4       0    NaN      NaN   NaN      NaN   

      penalty  other_wicket_type  ...  player_dismissed_T Kohler-Cadmore  \
0         N

In [41]:
print(Y)

0       1416472
1       1416472
2       1416472
3       1416472
4       1416472
         ...   
7790    1416505
7791    1416505
7792    1416505
7793    1416505
7794    1416505
Name: match_id, Length: 7795, dtype: int64


Train-Test Split

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [47]:
print(X.shape, X_train.shape, X_test.shape)

(7795, 399) (6236, 399) (1559, 399)


In [55]:
train_x = X_train.iloc()[: 1:]
test_x  = X_test.iloc()[: 1:]

train_y = Y_train.iloc()[: 0]
test_y  = Y_test.iloc()[: 0]

In [56]:
train_x.head(2)

,innings,ball,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,other_wicket_type,...,player_dismissed_T Kohler-Cadmore,player_dismissed_TL Seifert,player_dismissed_Tayyab Tahir,player_dismissed_Ubaid Shah,player_dismissed_Usama Mir,player_dismissed_Usman Khan,player_dismissed_Usman Tariq,player_dismissed_Waqar Salamkheil,player_dismissed_Yasir Khan,player_dismissed_Zaman Khan
121,2,1.1,4,0,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


In [57]:
train_y.head(2)

Series([], Name: match_id, dtype: int64)

In [58]:
test_x.head(2)

,innings,ball,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,other_wicket_type,...,player_dismissed_T Kohler-Cadmore,player_dismissed_TL Seifert,player_dismissed_Tayyab Tahir,player_dismissed_Ubaid Shah,player_dismissed_Usama Mir,player_dismissed_Usman Khan,player_dismissed_Usman Tariq,player_dismissed_Waqar Salamkheil,player_dismissed_Yasir Khan,player_dismissed_Zaman Khan
4546,2,0.1,6,0,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


In [59]:
test_y.head(2)

Series([], Name: match_id, dtype: int64)

 Applying SVM Classifier

In [75]:
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, Y_train)

ValueError: Input X contains NaN.
SVC does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [76]:
y_pred = svm_model.predict(X_test)

ValueError: Input X contains NaN.
SVC does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [79]:
accuracy = accuracy_score(Y_test, y_pred)
print(f"\nAccuracy of SVM Classifier: {accuracy}")
print("\nClassification Report:")
print(classification_report(Y_test, y_pred))


NameError: name 'y_pred' is not defined